**Data**

Below represents sample coupon data for a retailer. Each column (tokenized value) is extracted from OfferDetails
1. Product - Can be a single value or an array
2. FaceValue - The savings on the offer , generally a number
3. UOM - Unit of measure, represents the measuring unit for the product

In [1]:
import pandas as pd
import re
import nltk
import numpy as np

In [2]:
df = pd.read_csv('coupons_ner.csv', names = ['OfferDetails'])
print(df.shape)
df.head()

(886, 1)


,OfferDetails
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...


**Exercise 1**

1. Write a function that takes OfferDetails as input, and returns FaceValue as output
2. Run the function against all rows in the attached data and determine function accuracy (Higher the better)

*Hint*: For Exercise 1, regex function should do (ensure that all edge cases are taken care of)

In [3]:
def FaceVal(x):
    l = []
    s = x.replace(',',' ').split()
    for i in s:
        if '$' in i or '¢' in i:
            if '¢' in i:
                n = i.split('¢')[0]
                if n!= '':
                    i = str(float(n)/100)
                else: i = '0'
            i = '.'.join(re.sub('[^A-Za-z0-9]+',' ',i).split())
            l.append(i)
    if len(l) == 0 or l==['']:
        return ''
    elif len(l)>=1:
        temp = 0
        for k in l:
            if k !='':
                temp+= float((k.split('$')[-1]))
        if len(str(temp))==3:
            return '$'+str(temp)+'0'
        else:
            return '$'+str(temp)

In [4]:
# Test cases
c = ['','s', 's50','$$', '¢¢', '50$', '¢¢50', '$2.0','$2 50¢','$2 50 ¢','$2 50 50¢)))', '$2 50 50¢))) $$$$05,.m 50 150¢']
for i in c:
    print(i,':',FaceVal(i))
    print('-'*50)

 : 
--------------------------------------------------
s : 
--------------------------------------------------
s50 : 
--------------------------------------------------
$$ : 
--------------------------------------------------
¢¢ : $0.00
--------------------------------------------------
50$ : $50.0
--------------------------------------------------
¢¢50 : $0.00
--------------------------------------------------
$2.0 : $2.00
--------------------------------------------------
$2 50¢ : $2.50
--------------------------------------------------
$2 50 ¢ : $2.00
--------------------------------------------------
$2 50 50¢))) : $2.50
--------------------------------------------------
$2 50 50¢))) $$$$05,.m 50 150¢ : $9.00
--------------------------------------------------


Comment: These test cases indicate that the FaceVal function is good enough for extracting Face Value from each text.

In [6]:
df['FaceValue'] = df['OfferDetails'].map(FaceVal)
print(df.head())

                                        OfferDetails FaceValue
0  Save $2.00 ONE Downy Liquid Fabric Conditioner...     $2.00
1  Save $2.00 ONE Tide PODS OR Tide Power PODS (e...     $2.00
2  Save $2.00 ONE Tide Laundry Detergent (exclude...     $2.00
3  SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...     $1.00
4  $3.00 OFF when you purchase any THREE (3) Pepp...     $3.00


**Exercise 2**
1. Write a function that takes OfferDetails as input, and returns Product as output (Product can be single or array)
2. Run the function against all rows in the attached data and determine function accuracy (Higher the better)

*Hint*: For Exercise 2, build a corpus of products (built manually) and go from there. Mention the order of complexity (in any measures Big O, memory, cpu etc.). For Exercise 2, our expectation is that you implement NER model